In [3]:
import torch
import multiModel as mm
from multiModel import MultiInputModel
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Ścieżki i parametry
path_results = "training_results_224_2"
models_list = ['efficientnet_b0', 'efficientnet_v2_m', 'mobilenet_v3_small', 'resnet34']#, 'swin_t', 'vit_b_16']
batch_sizes = {
    'efficientnet_b0': 64,      #32,
    'mobilenet_v3_small': 64,   #32,
    'efficientnet_v2_m': 64,    #,16,
    'resnet34': 64,             #32,
    'swin_t': 32,
    'vit_b_16': 16
}
num_classes = 11
class_names = [str(i) for i in range(num_classes)]  # Zaktualizuj z rzeczywistymi nazwami klas, jeśli dostępne

# Wczytanie danych testowych
test_dataset = mm.MultiInputDataset("CSV/dataset/test_224.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)

# Funkcja do wczytywania logów
def load_logs(log_file):
    df = pd.read_csv(log_file)
    return df

# Funkcja do obliczania metryk
def calculate_metrics(true_labels, pred_labels):
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average="weighted")
    recall = recall_score(true_labels, pred_labels, average="weighted")
    f1 = f1_score(true_labels, pred_labels, average="weighted")
    return accuracy, precision, recall, f1

# Funkcja do testowania modelu
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    test_true = []
    test_pred = []
    start_time = time.time()
    with torch.no_grad():
        for t_image, b_image, s_image, labels in test_loader:
            t_image, b_image, s_image, labels = (
                t_image.to(device),
                b_image.to(device),
                s_image.to(device),
                labels.to(device)
            )
            outputs = model(t_image, b_image, s_image)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            test_true.extend(labels.cpu().numpy())
            test_pred.extend(predicted.cpu().numpy())
    end_time = time.time()
    test_loss /= len(test_loader)
    inference_time = end_time - start_time
    return test_loss, test_true, test_pred, inference_time

# Funkcja do mierzenia zużycia pamięci GPU
def get_gpu_memory():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**2  # MB
    return 0

# Funkcja do liczenia parametrów modelu
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Inicjalizacja raportu
report = {
    "Model": [],
    "Test Loss": [],
    "Test Accuracy": [],
    "Test Precision": [],
    "Test Recall": [],
    "Test F1": [],
    "Inference Time (s)": [],
    "Inference Time per Sample (ms)": [],
    "GPU Memory (MB)": [],
    "Parameters": []
}

# Pętla po modelach
for model_name in models_list:
    print(f"Processing {model_name}...")
    # Wczytanie logów
    log_file = f"{path_results}/training_log_{model_name}.txt"
    logs = load_logs(log_file)
    # Znalezienie najlepszej epoki na podstawie val_loss
    best_epoch = logs.loc[logs['val_loss'].idxmin()]
    print(f"Best epoch for {model_name}: {best_epoch['epoch']}, Val Loss: {best_epoch['val_loss']:.4f}")

    # Wczytanie najlepszego modelu
    best_model_path = f"{path_results}/best_model_{model_name}.pth"
    model = torch.load(best_model_path)
    model = model.to("cuda")
    model.eval()

    # Ustawienie batch_size
    batch_size = batch_sizes[model_name]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testowanie modelu
    criterion = nn.CrossEntropyLoss()
    test_loss, test_true, test_pred, inference_time = test_model(model, test_loader, criterion, "cuda")

    # Obliczenie metryk
    accuracy, precision, recall, f1 = calculate_metrics(test_true, test_pred)

    # Obliczenie czasu inferencji na próbkę
    num_samples = len(test_dataset)
    inference_time_per_sample = (inference_time / num_samples) * 1000  # ms

    # Pomiar zużycia pamięci GPU
    gpu_memory = get_gpu_memory()

    # Liczenie parametrów
    num_params = count_parameters(model)

    # Dodanie do raportu
    report["Model"].append(model_name)
    report["Test Loss"].append(test_loss)
    report["Test Accuracy"].append(accuracy)
    report["Test Precision"].append(precision)
    report["Test Recall"].append(recall)
    report["Test F1"].append(f1)
    report["Inference Time (s)"].append(inference_time)
    report["Inference Time per Sample (ms)"].append(inference_time_per_sample)
    report["GPU Memory (MB)"].append(gpu_memory)
    report["Parameters"].append(num_params)

    # Zwalnianie pamięci
    del model
    torch.cuda.empty_cache()

# Tworzenie DataFrame z raportem
report_df = pd.DataFrame(report)

# Sortowanie raportu po Test Accuracy
report_df = report_df.sort_values(by="Test Accuracy", ascending=False)

# Wyświetlenie raportu
print("\nComparative Report:")
print(report_df.to_string(index=False))

# Zapisanie raportu do pliku CSV
report_csv_path = f"{path_results}/comparative_report_batch64.csv"
report_df.to_csv(report_csv_path, index=False)
print(f"\nReport saved to {report_csv_path}")

Processing efficientnet_b0...
Best epoch for efficientnet_b0: 14.0, Val Loss: 0.0687


/tmp/ipykernel_99485/491750611.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)


Processing efficientnet_v2_m...
Best epoch for efficientnet_v2_m: 20.0, Val Loss: 0.0980


/tmp/ipykernel_99485/491750611.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)


Processing mobilenet_v3_small...
Best epoch for mobilenet_v3_small: 8.0, Val Loss: 0.1225


/tmp/ipykernel_99485/491750611.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)


Processing resnet34...
Best epoch for resnet34: 22.0, Val Loss: 0.1361


/tmp/ipykernel_99485/491750611.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)



Comparative Report:
             Model  Test Loss  Test Accuracy  Test Precision  Test Recall  Test F1  Inference Time (s)  Inference Time per Sample (ms)  GPU Memory (MB)  Parameters
   efficientnet_b0   0.069377       0.981860        0.981935     0.981860 0.981808           76.491440                        5.139172        29.054199     5393383
 efficientnet_v2_m   0.096431       0.976619        0.976762     0.976619 0.976598          125.810054                        8.452705       221.755859    54244191
mobilenet_v3_small   0.109049       0.968288        0.968789     0.968288 0.968320           65.626962                        4.409229        14.304199     1591947
          resnet34   0.130095       0.967415        0.967532     0.967415 0.967436           79.708193                        5.355294        92.095703    21884075

Report saved to training_results_224_2/comparative_report_batch64.csv


In [4]:
import torch
import multiModel as mm
from multiModel import MultiInputModel
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Ścieżki i parametry
path_results = "training_results_80_224_2"
models_list = ['mm64']
batch_sizes = {
    'efficientnet_b0': 64,      #32,
    'mobilenet_v3_small': 64,   #32,
    'efficientnet_v2_m': 64,    #,16,
    'resnet34': 64,             #32,
    'swin_t': 32,
    'vit_b_16': 16,
    'mm64': 64
}
num_classes = 11
class_names = [str(i) for i in range(num_classes)]  # Zaktualizuj z rzeczywistymi nazwami klas, jeśli dostępne

# Wczytanie danych testowych
test_dataset = mm.MultiInputDataset("CSV/dataset/test_80.csv", transform_rgb=mm.transform_rgb_224, transform_binary=mm.transform_binary_224)

# Funkcja do wczytywania logów
def load_logs(log_file):
    df = pd.read_csv(log_file)
    return df

# Funkcja do obliczania metryk
def calculate_metrics(true_labels, pred_labels):
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average="weighted")
    recall = recall_score(true_labels, pred_labels, average="weighted")
    f1 = f1_score(true_labels, pred_labels, average="weighted")
    return accuracy, precision, recall, f1

# Funkcja do testowania modelu
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    test_true = []
    test_pred = []
    start_time = time.time()
    with torch.no_grad():
        for t_image, b_image, s_image, labels in test_loader:
            t_image, b_image, s_image, labels = (
                t_image.to(device),
                b_image.to(device),
                s_image.to(device),
                labels.to(device)
            )
            outputs = model(t_image, b_image, s_image)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            test_true.extend(labels.cpu().numpy())
            test_pred.extend(predicted.cpu().numpy())
    end_time = time.time()
    test_loss /= len(test_loader)
    inference_time = end_time - start_time
    return test_loss, test_true, test_pred, inference_time

# Funkcja do mierzenia zużycia pamięci GPU
def get_gpu_memory():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**2  # MB
    return 0

# Funkcja do liczenia parametrów modelu
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Inicjalizacja raportu
report = {
    "Model": [],
    "Test Loss": [],
    "Test Accuracy": [],
    "Test Precision": [],
    "Test Recall": [],
    "Test F1": [],
    "Inference Time (s)": [],
    "Inference Time per Sample (ms)": [],
    "GPU Memory (MB)": [],
    "Parameters": []
}

# Pętla po modelach
for model_name in models_list:
    print(f"Processing {model_name}...")
    # Wczytanie logów
    log_file = f"{path_results}/training_log_{model_name}.txt"
    logs = load_logs(log_file)
    # Znalezienie najlepszej epoki na podstawie val_loss
    best_epoch = logs.loc[logs['val_loss'].idxmin()]
    print(f"Best epoch for {model_name}: {best_epoch['epoch']}, Val Loss: {best_epoch['val_loss']:.4f}")

    # Wczytanie najlepszego modelu
    best_model_path = f"{path_results}/best_model_{model_name}.pth"
    model = torch.load(best_model_path)
    model = model.to("cuda")
    model.eval()

    # Ustawienie batch_size
    batch_size = batch_sizes[model_name]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testowanie modelu
    criterion = nn.CrossEntropyLoss()
    test_loss, test_true, test_pred, inference_time = test_model(model, test_loader, criterion, "cuda")

    # Obliczenie metryk
    accuracy, precision, recall, f1 = calculate_metrics(test_true, test_pred)

    # Obliczenie czasu inferencji na próbkę
    num_samples = len(test_dataset)
    inference_time_per_sample = (inference_time / num_samples) * 1000  # ms

    # Pomiar zużycia pamięci GPU
    gpu_memory = get_gpu_memory()

    # Liczenie parametrów
    num_params = count_parameters(model)

    # Dodanie do raportu
    report["Model"].append(model_name)
    report["Test Loss"].append(test_loss)
    report["Test Accuracy"].append(accuracy)
    report["Test Precision"].append(precision)
    report["Test Recall"].append(recall)
    report["Test F1"].append(f1)
    report["Inference Time (s)"].append(inference_time)
    report["Inference Time per Sample (ms)"].append(inference_time_per_sample)
    report["GPU Memory (MB)"].append(gpu_memory)
    report["Parameters"].append(num_params)

    # Zwalnianie pamięci
    del model
    torch.cuda.empty_cache()

# Tworzenie DataFrame z raportem
report_df = pd.DataFrame(report)

# Sortowanie raportu po Test Accuracy
report_df = report_df.sort_values(by="Test Accuracy", ascending=False)

# Wyświetlenie raportu
print("\nComparative Report:")
print(report_df.to_string(index=False))

# Zapisanie raportu do pliku CSV
report_csv_path = f"{path_results}/comparative_report_batch64.csv"
report_df.to_csv(report_csv_path, index=False)
print(f"\nReport saved to {report_csv_path}")

Processing mm64...
Best epoch for mm64: 32.0, Val Loss: 0.2036


/tmp/ipykernel_99485/1488903860.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(best_model_path)



Comparative Report:
Model  Test Loss  Test Accuracy  Test Precision  Test Recall  Test F1  Inference Time (s)  Inference Time per Sample (ms)  GPU Memory (MB)  Parameters
 mm64   0.203917       0.936711        0.938295     0.936711 0.936575           37.242671                        2.502195         9.644043      397355

Report saved to training_results_80_224_2/comparative_report_batch64.csv
